In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

/home/wit/miniconda3/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
dax = pd.read_csv("./data/trade/dax_01_dir_daytype.csv", parse_dates=["date"])

In [5]:
cp = dax[["ibdir", "daydir"]].copy()

In [19]:
X_ = cp.ibdir
y_ = cp.daydir
encoder = LabelEncoder()
X = pd.get_dummies(X_)
y = encoder.fit_transform(y_)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)


In [20]:
class Fitter:
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
    def fit_model(self, m):
        ret = dict()
        m.fit(self.X_train, self.y_train)
        ret["train_acc"] = accuracy_score(m.predict(self.X_train), self.y_train)
        ret["test_acc"] = accuracy_score(m.predict(self.X_test),self.y_test)
        return (m, ret)


In [21]:
f = Fitter(X_train, y_train, X_test, y_test)

In [22]:
f.fit_model(LogisticRegression())

(LogisticRegression(), {'train_acc': 0.6717171717171717, 'test_acc': 0.76})

In [23]:
f.fit_model(DecisionTreeClassifier())

(DecisionTreeClassifier(), {'train_acc': 0.6717171717171717, 'test_acc': 0.76})

In [24]:
lr = f.fit_model(LogisticRegression())[0]

In [85]:
def prepare(da, sess_start, ib_end, sess_end):
    # Daily data
    d = da.copy()
    d.reset_index(inplace=True)
    d["date"] = d.datetime.dt.date
    open = d[d.datetime.dt.time == sess_start].open
    ibc = d[d.datetime.dt.time == ib_end].close
    df = pd.DataFrame((open, ibc), index=d.index,columns=["open", "ibc"])
    
    
    return df

In [86]:
from tvDatafeed import TvDatafeed, Interval
from datetime import time

In [87]:
tv = TvDatafeed()
last_10_bars = tv.get_hist("DAX", "XETR", Interval.in_30_minute, 18 * 10)
data = last_10_bars

In [88]:
prepare(data, time(9,0), time(10,0), time(17, 30))

ValueError: Length of values (2) does not match length of index (180)

In [60]:
last_10_bars

,symbol,open,high,low,close,volume
datetime,,,,,,
2022-03-23 09:00:00,XETR:DAX,14532.84,14539.31,14480.20,14499.61,325167266.0
2022-03-23 09:30:00,XETR:DAX,14499.90,14528.94,14488.37,14510.93,159130252.0
2022-03-23 10:00:00,XETR:DAX,14510.95,14521.01,14463.16,14468.70,131088064.0
2022-03-23 10:30:00,XETR:DAX,14468.42,14468.42,14401.74,14432.77,173610865.0
2022-03-23 11:00:00,XETR:DAX,14432.95,14443.92,14420.64,14420.64,114001329.0
...,...,...,...,...,...,...
2022-04-05 15:30:00,XETR:DAX,14410.01,14460.83,14410.01,14441.05,229842865.0
2022-04-05 16:00:00,XETR:DAX,14440.81,14459.72,14347.33,14365.99,286841738.0
2022-04-05 16:30:00,XETR:DAX,14365.66,14415.89,14355.63,14403.48,249089603.0
